In [ ]:
import numpy as np
import h5py
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.animation as animation
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.stats import gaussian_kde
from ipywidgets import interact, IntSlider
import seaborn as sns
from matplotlib.colors import LogNorm
import os
import src
from time import time

import ROOT
import tdrstyle
#import mplhep as hep
#hep.set_style(hep.style.CMS)

#pip install mplhep

In [ ]:
#hep.style.plt_style

## Set inputs

Optimal inputs for efficiency analysis:
- n_bins = 100
- q_max = 100000
- q_scaling = 1e9/3.61 ([source](https://cmssdt.cern.ch/lxr/source/SimTracker/SiPixelDigitizer/plugins/SiPixelDigitizerAlgorithm.cc#0202))

In [ ]:
# chunk size
chunk_size = 1000

# make a histogram with n bins
n_bins = 100

# charge maximum cut in [e-]
q_max = 100000

# tof maximum cut in [ns]
tof_max = 80

#tof scaling
tof_scaling = 1 #1e-9

#q scaling
q_scaling = 1e9/3.61  # 1e9/3.61 # from GeV to eV, from eV to e (3.6eV / e- in Si)

#q threshold
q_threshold = 1000

# window size: extend by this many in both directions
shift_n_times = 0

# tof offset in [ns]
shift_offset = 25

# print infos during run
verbose = False

# width of tornado mask [ns]
mask_xwidth = 25

#shift mask along x axis [ns]
mask_xoffset = 36-0.9113 # chose such that the whole mask is inside the [-20, 80] ns range

#set to -1 to mirror curves against time; stretch mask along x axis
mask_xscale = -1 

# to handle numerical rounding errors
num_precision= 8
epsilon = 1e-8

# remote path to hdf5
remote_path = "../../../sshfs/"

In [ ]:
tof_low_global = -20 - shift_n_times*shift_offset
tof_high_global = tof_max + shift_n_times*shift_offset
q_low_global = 0
q_high_global = q_max

# no epsilon
xedges_global = np.linspace(tof_low_global, tof_high_global, (2*shift_n_times+1)*n_bins+1)
yedges_global = np.linspace(q_low_global, q_high_global, n_bins+1)

For BIB the setting __beam 1__ is used. This means that the BIB particles come from the +Z side from the interface plane between the LHC long straight section and the CMS cavern, at Z=2260 cm. Shower will be produced on the -Z side due to interactions with the CMS detector material, therefore TEPX discs on the negative side will have a larger number of hits. We are interested in the __incoming__ BIB not the shower, so take __disc 4__ that is on the +Z side and is reached first by the BIB particles and therefore has __negative time of flight__ values (~-8 ns). For PU samples both disc 4 and -4 have a similar number of hits as the CMS geometry is symmetric. <br>
<img src="images/LHC.png" width="500"> ([source](https://sviret.web.cern.ch/sviret/Images/CMS/MIB/MIB/Welcome.php?n=Work.Gen))

In [ ]:
"""
simhit-simhit local
"""

hf_bib_halo = h5py.File('/Users/pkicsiny/bril_work/bib_simulations/hdf5/full_stat_simhit/tof_q_halo.h5', 'r')
hf_bib_carbon = h5py.File('/Users/pkicsiny/bril_work/bib_simulations/hdf5/full_stat_simhit/tof_q_gas_carbon.h5', 'r')
hf_bib_oxygen = h5py.File('/Users/pkicsiny/bril_work/bib_simulations/hdf5/full_stat_simhit/tof_q_gas_oxygen.h5', 'r')
hf_bib_hydrogen = h5py.File('/Users/pkicsiny/bril_work/bib_simulations/hdf5/full_stat_simhit/tof_q_gas_hydrogen.h5', 'r')

hf_pu_dir = os.listdir("/Users/pkicsiny/bril_work/bib_simulations/hdf5/full_stat_simhit")
hf_pu_files = [f for f in hf_pu_dir if ".h5" in f and "tof_q_PU" in f]
hf_pu_list = [] 
for hf_pu_file in hf_pu_files:
    print(hf_pu_file)
    hf_pu_list.append(h5py.File(os.path.join("/Users/pkicsiny/bril_work/bib_simulations/hdf5/full_stat_simhit", hf_pu_file), "r"))

# Read hdf5 files

In [ ]:
disc= 4#list(range(-4, 5))
ring= 1#list(range(-4, 5))

input_data = [hf_pu_list, hf_bib_halo, hf_bib_carbon, hf_bib_oxygen, hf_bib_hydrogen]
pu_dset_, bib_halo_dset_, bib_carbon_dset_, bib_oxygen_dset_, bib_hydrogen_dset_ = src.readH5Data(
    input_data, disc_list=[disc]*len(input_data), ring_list=[ring]*len(input_data), group_name='Tof_q')

bib_dset_ = np.vstack([bib_carbon_dset_, bib_oxygen_dset_, bib_hydrogen_dset_, bib_halo_dset_])

In [ ]:
# cut off high tof and high charge
bib_halo_dset = bib_halo_dset_[[p[0]<tof_max and p[1]<q_max for p in bib_halo_dset_]]
bib_carbon_dset = bib_carbon_dset_[[p[0]<tof_max and p[1]<q_max for p in bib_carbon_dset_]]
bib_oxygen_dset = bib_oxygen_dset_[[p[0]<tof_max and p[1]<q_max for p in bib_oxygen_dset_]]
bib_hydrogen_dset = bib_hydrogen_dset_[[p[0]<tof_max and p[1]<q_max for p in bib_hydrogen_dset_]]
pu_dset = pu_dset_[[p[0]<tof_max and p[1]<q_max for p in pu_dset_]]

print("Length of original BIB halo dataset: {}".format(len(bib_halo_dset_)))
print("Length of BIB halo dataset after clip: {} ({} %)".format(len(bib_halo_dset),
                           round(100*len(bib_halo_dset)/(len(bib_halo_dset_)+1e-8), 2)))

print("Length of original BIB carbon dataset: {}".format(len(bib_carbon_dset_)))
print("Length of BIB carbon dataset after clip: {} ({} %)".format(len(bib_carbon_dset),
                            round(100*len(bib_carbon_dset)/len(bib_carbon_dset_), 2)))

print("Length of original BIB oxygen dataset: {}".format(len(bib_oxygen_dset_)))
print("Length of BIB oxygen dataset after clip: {} ({} %)".format(len(bib_oxygen_dset),
                            round(100*len(bib_oxygen_dset)/len(bib_oxygen_dset_), 2)))

print("Length of original BIB hydrogen dataset: {}".format(len(bib_hydrogen_dset_)))
print("Length of BIB hydrogen dataset after clip: {} ({} %)".format(len(bib_hydrogen_dset),
                            round(100*len(bib_hydrogen_dset)/len(bib_hydrogen_dset_), 2)))

print("Length of original PU dataset: {}".format(len(pu_dset_)))
print("Length of PU dataset after clip: {} ({} %)".format(len(pu_dset), round(100*len(pu_dset)/len(pu_dset_), 2)))

In [ ]:
# PU
print("pu")
pu_tof_histo, pu_tof_below_histo, pu_tof_above_histo, pu_xedges, pu_yedges = src.binH5Data(pu_dset,
    n_bins=n_bins, chunk_size=chunk_size, tof_scaling=tof_scaling, q_scaling=q_scaling,
    q_threshold=q_threshold, shift_n_times=shift_n_times,shift_offset=shift_offset,
    epsilon=epsilon, verbose=verbose,
    xedges=xedges_global, yedges=yedges_global)


# BIB halo
print("halo")
bib_halo_tof_histo, bib_halo_tof_below_histo, bib_halo_tof_above_histo, bib_halo_xedges, bib_halo_yedges = src.binH5Data(bib_halo_dset,
    n_bins=n_bins, chunk_size=chunk_size, tof_scaling=1, q_scaling=q_scaling,
    q_threshold=q_threshold, shift_n_times=shift_n_times, shift_offset=shift_offset,
    epsilon=epsilon, verbose=verbose, 
    xedges=xedges_global, yedges=yedges_global)

# BIB carbon
print("carbon")
bib_carbon_tof_histo, bib_carbon_tof_below_histo, bib_carbon_tof_above_histo, bib_carbon_xedges, bib_carbon_yedges = src.binH5Data(bib_carbon_dset,
    n_bins=n_bins, chunk_size=chunk_size, tof_scaling=1, q_scaling=q_scaling,
    q_threshold=q_threshold, shift_n_times=shift_n_times, shift_offset=shift_offset,
    epsilon=epsilon, verbose=verbose, 
    xedges=xedges_global, yedges=yedges_global)

# BIB oxygen
print("oxygen")
bib_oxygen_tof_histo, bib_oxygen_tof_below_histo, bib_oxygen_tof_above_histo, bib_oxygen_xedges, bib_oxygen_yedges = src.binH5Data(bib_oxygen_dset,
    n_bins=n_bins, chunk_size=chunk_size, tof_scaling=1, q_scaling=q_scaling,
    q_threshold=q_threshold, shift_n_times=shift_n_times, shift_offset=shift_offset,
    epsilon=epsilon, verbose=verbose, 
    xedges=xedges_global, yedges=yedges_global)
    
# BIB hydrogen
print("hydrogen")
bib_hydrogen_tof_histo, bib_hydrogen_tof_below_histo, bib_hydrogen_tof_above_histo, bib_hydrogen_xedges, bib_hydrogen_yedges = src.binH5Data(bib_hydrogen_dset,
    n_bins=n_bins, chunk_size=chunk_size, tof_scaling=1, q_scaling=q_scaling,
    q_threshold=q_threshold, shift_n_times=shift_n_times, shift_offset=shift_offset,
    epsilon=epsilon, verbose=verbose,
    xedges=xedges_global, yedges=yedges_global)

#assert np.sum(pu_yedges == bib_yedges) == n_bins+1, "Histogram bins are not the same"

# sum up bib types
bib_dset = np.vstack([bib_carbon_dset, bib_oxygen_dset, bib_hydrogen_dset, bib_halo_dset])
bib_tof_histo = bib_carbon_tof_histo + bib_oxygen_tof_histo + bib_hydrogen_tof_histo + bib_halo_tof_histo
bib_tof_above_histo = bib_carbon_tof_above_histo + bib_oxygen_tof_above_histo + bib_hydrogen_tof_above_histo + bib_halo_tof_above_histo
bib_tof_below_histo = bib_carbon_tof_below_histo + bib_oxygen_tof_below_histo + bib_hydrogen_tof_below_histo + bib_halo_tof_below_histo

# bib gas total
bib_gas_dset = np.vstack([bib_carbon_dset, bib_oxygen_dset, bib_hydrogen_dset])
bib_gas_tof_histo = bib_carbon_tof_histo + bib_oxygen_tof_histo + bib_hydrogen_tof_histo
bib_gas_tof_above_histo = bib_carbon_tof_above_histo + bib_oxygen_tof_above_histo + bib_hydrogen_tof_above_histo
bib_gas_tof_below_histo = bib_carbon_tof_below_histo + bib_oxygen_tof_below_histo + bib_hydrogen_tof_below_histo

# 2D histos one by one

In [ ]:
# PU and BIB on same plot as scatterplots
#plt.style.use(hep.style.CMS)
plt.figure(figsize=(10,10))
plt.subplots_adjust(left=0.2)


x_bib = list(map(lambda x: x[0], bib_oxygen_dset))
y_bib = list(map(lambda x: x[1]*q_scaling, bib_oxygen_dset))
values_bib = np.vstack((x_bib, y_bib))
kernel_bib = gaussian_kde(values_bib)
kde_bib = kernel_bib.evaluate(values_bib)
norm_bib = Normalize(vmin=kde_bib.min(), vmax=kde_bib.max())
colors_bib = cm.ScalarMappable(norm=norm_bib, cmap='jet').to_rgba(kde_bib)
plt.scatter(x_bib, y_bib,
           label="BIB entries: {}".format(len(bib_oxygen_dset)), s=2, c=colors_bib)


x_pu = list(map(lambda x: x[0], pu_dset))
y_pu = list(map(lambda x: x[1]*q_scaling, pu_dset))
values_pu = np.vstack((x_pu, y_pu))
kernel_pu = gaussian_kde(values_pu)
kde_pu = kernel_pu.evaluate(values_pu)
norm_pu = Normalize(vmin=kde_pu.min(), vmax=kde_pu.max())
colors_pu = cm.ScalarMappable(norm=norm_pu, cmap='viridis').to_rgba(kde_pu)
plt.scatter(x_pu, y_pu,
            label="PU entries: {}".format(len(pu_dset)), s=2, c=colors_pu)

lgnd = plt.legend(markerscale=10)
plt.grid()
plt.xlabel("Time of flight [ns]", fontsize=20)
plt.ylabel("Induced charge [e-]", fontsize=20)
plt.title(r"$\bf{Phase-2}$ Simulation Preliminary", fontsize=24, loc="left")

# Just experiment label and <text> such as 'Preliminary' or 'Simulation'
#hep.cms.text("Phase-2 Simulation Preliminary")
plt.ylim(0, 80000)
plt.xlim(-20, 80)
#plt.savefig("scatter_low_stat_rechit_digi_notimewalk_fine.png")
#plt.savefig("adc_study/dist_{}.png".format(study))

In [ ]:
#plt.style.use(hep.style.CMS)

plt.figure(figsize=(10,10))
plt.imshow(np.ma.masked_where(bib_halo_tof_histo==0, bib_halo_tof_histo), interpolation='nearest', origin='lower',
        extent=[xedges_global[0], xedges_global[-1], yedges_global[0], yedges_global[-1]],
        aspect="auto")
plt.grid()
plt.xlabel("bib_halo_tof [ns]", fontsize=20)
plt.ylabel("Q [e-]", fontsize=20)
#plt.ylim(0, 100000)
#plt.xlim(-30, 60)
plt.title("halo gas: disc {}, ring {}, {} entries".format(
    disc, ring, int(np.sum(bib_halo_tof_histo))), fontsize=20)

plt.subplots_adjust(left=0.15)
plt.colorbar()
#plt.savefig("bib_gas_halo.png")

In [ ]:
#plt.style.use(hep.style.CMS)

plt.figure(figsize=(10,10))
plt.imshow(np.ma.masked_where(bib_gas_tof_histo==0, bib_gas_tof_histo), interpolation='nearest', origin='lower',
        extent=[xedges_global[0], xedges_global[-1], yedges_global[0], yedges_global[-1]],
        aspect="auto")
plt.grid()
plt.xlabel("bib_carbon_tof [ns]", fontsize=20)
plt.ylabel("Q [e-]", fontsize=20)
#plt.ylim(0, 100000)
#plt.xlim(-30, 60)
plt.title("Carbon gas: disc {}, ring {}, {} entries".format(
    disc, ring, int(np.sum(bib_gas_tof_histo))), fontsize=20)

plt.subplots_adjust(left=0.15)
plt.colorbar()
#plt.savefig("bib_gas_hydrogen.png")

In [ ]:
#plt.style.use(hep.style.CMS)

plt.figure(figsize=(10,10))
plt.imshow(np.ma.masked_where(pu_tof_histo==0, pu_tof_histo), interpolation='nearest', origin='lower',
        extent=[pu_xedges[0], pu_xedges[-1], pu_yedges[0], pu_yedges[-1]], aspect="auto")
plt.grid()
plt.xlabel("pu_tof [ns]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0, 100000)
plt.title("Pileup: disc {}, Ring {}, {} entries".format(disc, ring, int(np.sum(pu_tof_histo))), fontsize=20)
plt.colorbar()
#plt.savefig("pu_test.png")

# Load timewalk sim data

Simulated time of flight vs. induced charge curves for the TEPX readout chip (RD53B). Models the timewalk effect in the chip that tells the delay in the signal detection depending on the charge, given a constant threshold discriminator. In general, low charges with small signals are detected later than higher charges with large signals due to their slower rise as seen on the figure below. Signals that reach the threshold after 25 ns (between the blue and red curves) will be assigned to the next bunch crossing by the readout electronics. <br>
<img src="images/timewalk.png" width="500" /> <br>
([source](https://indico.cern.ch/event/818375/contributions/3430925/attachments/1844903/3026483/TrackerWeek_LateHitAnalysis.pdf))

In [ ]:
#read timewalk sim
data = pd.read_csv("../tornado_plot/timewalk_Qth1000_RD53B.csv", header=0)

# insert a 0th and a n+1th row to facilitate thresholding at 1000 e-
data = pd.concat([pd.DataFrame(data.iloc[0]).T, data]).reset_index(drop=True)

# convert from seconds to nanoseconds
data["Timewalk"] *= 1e9

In [ ]:
data.at[0, "Timewalk"] = data.at[0, "Timewalk"] + mask_xwidth*tof_scaling
data = pd.concat([data, pd.DataFrame(data.iloc[-1]).T]).reset_index(drop=True)
data.at[len(data)-1, "Qel "] = q_high_global # data.at[len(data)-1, "Qel "]*2

#scale mask e.g. mirror in time
data["Timewalk"] = mask_xscale*data["Timewalk"] + mask_xoffset*tof_scaling

#shift curve to get other boundary
data["timewalk_shifted"] = data["Timewalk"] + mask_xwidth*tof_scaling
data

In [ ]:
#plot
fig = plt.figure(figsize=(10, 10))
for i in range(-shift_n_times, shift_n_times+1):
    l1 = "simulation" if i == 0 else ""
    l2 = "shifted by {}ns".format(mask_xwidth) if i == 0 else ""
    plt.plot(data["Timewalk"]+i*shift_offset*tof_scaling, data["Qel "], label=l1, c="b")
    plt.plot(data["timewalk_shifted"]+i*shift_offset*tof_scaling, data["Qel "], label=l2, c="r")
plt.xlabel("Time of flight [ns]")
plt.ylabel("Q [e-]")
plt.legend()
plt.grid()

# Overlay sim curve and bib data

In [ ]:
combined_tof_above_histo = bib_tof_above_histo + pu_tof_above_histo
combined_tof_below_histo = bib_tof_above_histo + pu_tof_above_histo
combined_tof_histo = bib_tof_histo + pu_tof_histo
overlay_xedges = xedges_global
overlay_yedges = yedges_global

In [ ]:
X_mask, Y_mask = np.meshgrid(overlay_xedges, overlay_yedges)

In [ ]:
plt.figure(figsize=(10, 10))
plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(combined_tof_above_histo==0, combined_tof_above_histo))
#plt.ylim(0, .01*q_scaling)
#plt.xlim(8, 13)
plt.grid()

# tornado mask curves
plt.plot(data["Timewalk"], data["Qel "], label="simulation")
plt.plot(data["timewalk_shifted"], data["Qel "], label="shifted", c="r")
plt.legend()
plt.xlabel("Time of flight [ns]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)

In [ ]:
# Landau distribution
#plt.plot(np.sum(pu_tof_above_histo,axis=1))
plt.plot(np.sum(bib_tof_above_histo,axis=1))

# Coordinate geometry way to calculate area below curve
Need to run this only once to get the mask bound to the binning. <br>
Use 2 separate masks one for BIB one for PU, instead of one composite. <br>
Optimization of chip efficiency is done afterwards with steps of one bin width. <br>
Goal is to measure incoming BIB using the first 25 ns window (first mask) while measuring PU with the following 25 ns window. <br>
Clear BIB measurement is only possible if we use the first bunch crossing in every bunch train as BIB particles coming later will be superimposed on hit collisions therefore it becomes impossible to tell apart BIB from PU hits.

In [ ]:
verbose=False

In [ ]:
cx_list = []
cy_list = []

cell_mask_pu = np.zeros((n_bins, (2*shift_n_times+1)*n_bins))#*np.nan
cell_mask_bib = np.zeros((n_bins, (2*shift_n_times+1)*n_bins))#*np.nan

# measure elapsed time
start = time()

# loop goes from 0 to nbins, starts from bottom left corner
# loop over rows
for yi in range(n_bins):
    print("Bin row [{}/{}]".format(yi+1, n_bins))
    
    # loop over columns
    for xi in range((2*shift_n_times+1)*n_bins):
        xmin, xmax = round(overlay_xedges[xi], num_precision), round(overlay_xedges[xi+1], num_precision)
        ymin, ymax = round(overlay_yedges[yi], num_precision), round(overlay_yedges[yi+1], num_precision)
        cell_area = round((xmax - xmin)*(ymax - ymin), num_precision)         
        cx = round((xmax + xmin)/2, num_precision)
        cy = round((ymax + ymin)/2, num_precision)
        
        # loop over masks in order: bib (-25ns offset), pu (0ns offset)
        for mask_idx in range(-1, 1):
            
            """
            place these 2 lines outside for loop if one composite mask is used
            """
            area_below_left_curve = []
            area_below_right_curve = []
        
            if verbose:
                print("working on mask copy {}".format(mask_idx))
                
            #get cell area below all curves
            area_below_left_curve.append(src.getAreaBelowCurve(xmin, xmax, ymin, ymax, cx, cy,
                                        data["Timewalk"]+mask_idx*shift_offset*tof_scaling,
                                        data["Qel "], verbose=verbose))
            area_below_right_curve.append(src.getAreaBelowCurve(xmin, xmax, ymin, ymax, cx, cy,
                                        data["timewalk_shifted"]+mask_idx*shift_offset*tof_scaling,
                                        data["Qel "], verbose=verbose))
            """
            untab all lines below and remove last if if composite mask is used
            """
            # cell mask value is between 0 and 1
            area_between_curves = np.sum(area_below_left_curve) - np.sum(area_below_right_curve)
            cell_mask_value = area_between_curves/cell_area
            
            #fill up mask array with cell mask value
            if cell_mask_value < epsilon:
                cell_mask_value = 0
            else:
                
                #store cells that are between two curves
                cx_list.append(cx)
                cy_list.append(cy)
                
                #numerics correction
                if cell_mask_value > 1:
                    cell_mask_value = 1
                if np.abs(cell_mask_value - 1) < epsilon:
                    cell_mask_value = 1   
      
                #save cell masks into numpy array (start from top left corner)
                if mask_idx==-1: # bib mask
                    cell_mask_bib[yi, xi] = cell_mask_value
                elif mask_idx==0:
                    cell_mask_pu[yi, xi] = cell_mask_value
                else:
                    print("Something's fishy.")
        
end = time()
print("Elapsed time: {} s".format(end-start))

In [ ]:
#%matplotlib

#data
fig = plt.figure(figsize=(10, 10))
data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(combined_tof_above_histo==0, combined_tof_above_histo),
                           vmin=1, vmax=np.max(combined_tof_above_histo), norm=LogNorm())

# tornado mask bounding curves
for i in range(-1, 1):
    l1 = "simulation" if i == 0 else ""
    l2 = "shifted by {}ns".format(mask_xwidth) if i == 0 else ""
    plt.plot(data["Timewalk"]+i*shift_offset*tof_scaling, data["Qel "], label=l1, c="b")
    plt.plot(data["timewalk_shifted"]+i*shift_offset*tof_scaling, data["Qel "], label=l2, c="r")

#tornado mask
mask = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(cell_mask_bib==-1, cell_mask_bib),
                      alpha=.5, cmap=plt.get_cmap('Greys_r'))

plt.legend(facecolor='white', framealpha=1)
plt.xlabel("Time of flight [ns]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0,80000)
plt.xlim(xedges_global[0], xedges_global[-1])
#cbar = fig.colorbar(data_plot) 
#cbar.set_label('Data count',size=18)
plt.grid()

plt.axvline(data["Timewalk"].iloc[-1], label="mask coordinate", c="g")
plt.title("All hits [ {} hits| 100%]".format(int(np.sum(combined_tof_above_histo))))

# Apply mask to data and produce hit efficiency plots

In [ ]:
cell_mask_pu_list = []
cell_mask_bib_list = []
cell_mask_coord_list = []
mask_coord = data["Timewalk"].iloc[-1]
bin_width = int(xedges_global[-1] - xedges_global[0])/n_bins


mask_coord_bin = int(np.floor(np.abs(mask_coord - xedges_global[0])/bin_width))


# goes from 0 to 199
for i in range(2*n_bins-1):
    
    # get coordinate of tornado mask (original curve loaded from the file, not the 25ns shifted)
    mask_coord_i = mask_coord-bin_width*((n_bins-1)-i)
    cell_mask_coord_list.append(mask_coord_i)
    
    # crop mask (no cropping when i=n_bins-1)
    cell_mask_pu_cropped = cell_mask_pu[:,-(i+1):(2*n_bins-1)-i]
    cell_mask_bib_cropped = cell_mask_bib[:,-(i+1):(2*n_bins-1)-i]

    # pad mask (no padding when i=n_bins-1)
    if i <= n_bins-1:
        cell_mask_pu_pad = np.zeros((n_bins, (n_bins-1)-i))#*np.inf
        cell_mask_pu_padded = np.hstack((cell_mask_pu_cropped, cell_mask_pu_pad))
        
        cell_mask_bib_pad = np.zeros((n_bins, (n_bins-1)-i))#*np.inf
        cell_mask_bib_padded = np.hstack((cell_mask_bib_cropped, cell_mask_bib_pad))
    elif i > n_bins-1:
        cell_mask_pu_pad = np.zeros((n_bins, i-(n_bins-1)))#*np.inf
        cell_mask_pu_padded = np.hstack((cell_mask_pu_pad, cell_mask_pu_cropped))

        cell_mask_bib_pad = np.zeros((n_bins, i-(n_bins-1)))#*np.inf
        cell_mask_bib_padded = np.hstack((cell_mask_bib_pad, cell_mask_bib_cropped))

    print(i, "PU: Crop shape: {} Pad shape: {} Mask coord: {}".format(
        np.shape(cell_mask_pu_cropped), np.shape(cell_mask_pu_pad), mask_coord_i))
 
    print(i, "BIB: Crop shape: {} Pad shape: {} Mask coord: {}".format(
        np.shape(cell_mask_bib_cropped), np.shape(cell_mask_bib_pad), mask_coord_i))
 
    cell_mask_pu_list.append(cell_mask_pu_padded)
    cell_mask_bib_list.append(cell_mask_bib_padded)

In [ ]:
%matplotlib notebook

#animate mask overlay
n_frames = 2*n_bins

#init dictionaries for storing efficiencies
pu_hits_dict = {"alpha":[], "beta":[], "alpha_eff":[], "beta_eff":[], "alpha_histo":[], "beta_histo":[]}
bib_hits_dict = {"alpha":[], "beta":[], "alpha_eff":[], "beta_eff":[], "alpha_histo":[], "beta_histo":[]}

fig, ax = plt.subplots(figsize=(10,5)) # make figure
plt.subplots_adjust(left=0.2)
ax.grid()
plt.xlabel("Time of flight (ns)", fontsize=16)
plt.ylabel("Charge / {} ns (e-)".format(bin_width), fontsize=16)

"""
plot PU 200 + BIB data
"""
# init, show bib and tof separately with different color mapping ad on log scale (plot just once)
all_mesh = ax.pcolormesh(X_mask, Y_mask, np.ma.masked_where(combined_tof_histo==0, combined_tof_histo),
                               vmin=1, vmax=np.max(combined_tof_histo),
                               norm=LogNorm(), cmap=plt.get_cmap('viridis'))

"""
colorbar settings
"""
all_cbar = plt.colorbar(all_mesh)
all_cbar.ax.set_ylabel('Entries')

"""
animation
"""
image = ax.pcolormesh(X_mask, Y_mask, np.zeros_like(combined_tof_histo), cmap=plt.get_cmap('Greys_r'),
                alpha=.3, vmin=0, vmax=1)    
mask_boundary, = ax.plot([], [], c="r")

def animate_mask(i):
    # set the data in the axesimage object
    cell_mask_pu_current = cell_mask_pu_list[i]
    cell_mask_bib_current = cell_mask_bib_list[i]
    cell_mask_coord_current = cell_mask_coord_list[i]
    
    # need alpha hits for pu mask and beta hits for bib mask
    pu_alpha_hits = src.mcOverlay(cell_mask_pu_current, pu_tof_above_histo)
    pu_beta_hits = pu_tof_above_histo - pu_alpha_hits
    bib_alpha_hits = src.mcOverlay(cell_mask_bib_current, bib_tof_above_histo)
    bib_beta_hits = bib_tof_above_histo - bib_alpha_hits
    
    pu_hits_dict["alpha_histo"].append(pu_alpha_hits)
    bib_hits_dict["alpha_histo"].append(bib_alpha_hits)
    pu_hits_dict["beta_histo"].append(pu_beta_hits)
    bib_hits_dict["beta_histo"].append(bib_beta_hits)
    
    pu_hits_dict["alpha"].append(int(np.sum(pu_alpha_hits)))
    bib_hits_dict["alpha"].append(int(np.sum(bib_alpha_hits)))
    pu_hits_dict["beta"].append(int(np.sum(pu_beta_hits)))
    bib_hits_dict["beta"].append(int(np.sum(bib_beta_hits)))
    
    pu_hits_dict["alpha_eff"].append(100*pu_hits_dict["alpha"][-1]/np.sum(pu_tof_above_histo))
    bib_hits_dict["alpha_eff"].append(100*bib_hits_dict["alpha"][-1]/np.sum(bib_tof_above_histo))
    pu_hits_dict["beta_eff"].append(100*pu_hits_dict["beta"][-1]/np.sum(pu_tof_above_histo))
    bib_hits_dict["beta_eff"].append(100*bib_hits_dict["beta"][-1]/np.sum(bib_tof_above_histo))
    
    # update animation
    image.set_array(cell_mask_pu_current+cell_mask_bib_current)
    mask_boundary.set_data(data["Timewalk"]-bin_width*((n_bins-1)-i), data["Qel "])
    
    # update title
    fig_title = "Step {}, Mask coordinate: {} ns\nPU 200: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]\nBIB: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]".format(i,
        round(cell_mask_coord_current, 2),
        pu_hits_dict["alpha"][-1], round(pu_hits_dict["alpha_eff"][-1], 2),
        pu_hits_dict["beta"][-1], round(pu_hits_dict["beta_eff"][-1], 2),
        bib_hits_dict["alpha"][-1], round(bib_hits_dict["alpha_eff"][-1], 2),
        bib_hits_dict["beta"][-1], round(bib_hits_dict["beta_eff"][-1], 2))
    ax.set_title(fig_title, fontsize=12)

    return [image, mask_boundary]

#will make 2*n_bins-1 steps even if the frames is larger
ani = animation.FuncAnimation(fig, animate_mask, frames=n_frames,
                              interval=100, repeat=False)

#ani.save('mask_overlay.gif', writer='imagemagick', fps=30)


In [ ]:
for ii, jj in zip(pu_hits_dict.keys(), bib_hits_dict.keys()):
    pu_hits_dict[ii] = pu_hits_dict[ii][1:]
    bib_hits_dict[jj] = bib_hits_dict[jj][1:]

In [ ]:
alpha_tot = np.add(pu_hits_dict["alpha"], bib_hits_dict["alpha"])
beta_tot = np.add(pu_hits_dict["beta"], bib_hits_dict["beta"])
alpha_eff_tot = np.add(pu_hits_dict["alpha_eff"], bib_hits_dict["alpha_eff"])
beta_eff_tot = np.add(pu_hits_dict["beta_eff"], bib_hits_dict["beta_eff"])
max_eff = np.argmax(alpha_eff_tot)

In [ ]:
eff_range_min = 34
eff_range_max = 115

In [ ]:
%matplotlib inline
plt.ioff()

fix, ax = plt.subplots(2,1)
plt.tight_layout()

ax[0].scatter(cell_mask_coord_list[eff_range_min:eff_range_max],
              pu_hits_dict["alpha_eff"][eff_range_min:eff_range_max], label="PU 200 alpha", c="red", s=3)
ax[1].scatter(cell_mask_coord_list[eff_range_min:eff_range_max],
              pu_hits_dict["beta_eff"][eff_range_min:eff_range_max], label="PU 200 beta", c="red", s=3)
ax[0].scatter(cell_mask_coord_list[eff_range_min:eff_range_max],
              bib_hits_dict["alpha_eff"][eff_range_min:eff_range_max], label="BIB alpha", c="blue", s=3)
ax[1].scatter(cell_mask_coord_list[eff_range_min:eff_range_max],
              bib_hits_dict["beta_eff"][eff_range_min:eff_range_max], label="BIB beta", c="blue", s=3)

ax[0].axvline(cell_mask_coord_list[max_eff], label="Max. efficiency at {} ns".format(
    round(cell_mask_coord_list[max_eff], 4)), c="g", linestyle="--", linewidth=1)
ax[1].axvline(cell_mask_coord_list[max_eff], label="Max. efficiency at {} ns".format(
    round(cell_mask_coord_list[max_eff], 4)), c="g", linestyle="--", linewidth=1)


ax[0].set_ylim(-5,105)
ax[0].legend(loc="right")
ax[0].grid()
ax[1].set_ylim(-5,105)
ax[1].legend(loc="right")
ax[1].grid()

ax[0].set_ylabel("Efficiency / {} ns (%)".format(bin_width), fontsize=16)
ax[1].set_xlabel("Mask coordinate (ns)", fontsize=16)

#plt.savefig("efficiencies_test.png")
plt.show()

In [ ]:
plt.ioff()
fix, ax = plt.subplots(2,1)
ax[0].scatter(cell_mask_coord_list[:n_frames], alpha_eff_tot,
              label="Total alpha efficiency", c="k", s=3)
ax[1].scatter(cell_mask_coord_list[:n_frames], beta_eff_tot,
              label="Total beta efficiency", c="k", s=3)

ax[0].axvline(cell_mask_coord_list[max_eff], label="Max. efficiency at {} ns".format(
    round(cell_mask_coord_list[max_eff], 4)))
ax[1].axvline(cell_mask_coord_list[max_eff], label="Max. efficiency at {} ns".format(
    round(cell_mask_coord_list[max_eff], 4)))

ax[0].legend()
ax[0].grid()
ax[1].legend()
ax[1].grid()

ax[0].set_ylabel("Efficiency / {} ns (%)".format(bin_width), fontsize=16)
ax[1].set_xlabel("Mask coordinate (ns)", fontsize=16)
ax[0].set_title(r"$\bf{CMS\,\,Phase}\bf{-2}$ Simulation Preliminary", fontsize=16, loc="left")

#plt.savefig("efficiencies_test.png")
plt.show()

In [ ]:
#show plot of data and mask at right coordinate
#max_eff = 28
"""
plot PU 200 + BIB data
"""
fig = plt.figure(figsize=(9,6))
all_data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(combined_tof_above_histo==0, combined_tof_above_histo),
                              vmin=1, vmax=np.max(combined_tof_above_histo),
                              norm=LogNorm(), cmap=plt.get_cmap('viridis'))

#bib_data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(bib_tof_above_histo==0, bib_tof_above_histo),
#                               vmin=1, vmax=np.max(combined_tof_above_histo),
#                               norm=LogNorm(), cmap=plt.get_cmap('Greys_r'))

#pu_data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(pu_tof_above_histo==0, pu_tof_above_histo),
#                              vmin=1, vmax=np.max(combined_tof_above_histo),
#                              norm=LogNorm(), cmap=plt.get_cmap('viridis'))


"""
timewalk curve
"""
# shift timewalk curves to optimal position
diff = data["Timewalk"].iloc[-1]-cell_mask_coord_list[max_eff]

#plt.axvline(cell_mask_coord_list[max_eff], c="r")
plt.plot(data["timewalk_shifted"]-shift_offset*tof_scaling-diff, data["Qel "], label=l1, c="r")

"""
colorbar settings
"""
all_cbar = plt.colorbar(all_data_plot)
all_cbar.ax.set_ylabel('Entries')
#bib_cbar = plt.colorbar(bib_data_plot)
#bib_cbar.ax.set_ylabel('BIB entries')
#pu_cbar = plt.colorbar(pu_data_plot)
#pu_cbar.ax.set_ylabel('PU 200 entries')

"""
tornado mask at max. efficiency
"""
mask = plt.pcolormesh(X_mask, Y_mask, cell_mask_pu_list[max_eff]+cell_mask_bib_list[max_eff],
                      alpha=.5, cmap=plt.get_cmap('Greys_r'))

"""
plot settings
"""
plt.xlabel("Time of flight (ns)", fontsize=16)
plt.ylabel("Charge / {} ns (e-)".format(bin_width), fontsize=16)
#plt.ylim(0,80000)
plt.xlim(overlay_xedges[0], overlay_xedges[-1])
#cbar = fig.colorbar(data_plot) 
#cbar.set_label('Data count',size=18)
plt.grid()

fig_title = "Mask coordinate: {} ns\nPU 200: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]\nBIB: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]".format(
        round(cell_mask_coord_list[max_eff], 2),
        int(pu_hits_dict["alpha"][max_eff+1]), round(100*int(pu_hits_dict["alpha"][max_eff+1])/np.sum(pu_tof_above_histo), 2),
        int(pu_hits_dict["beta"][max_eff+1]), round(100*int(pu_hits_dict["beta"][max_eff+1])/np.sum(pu_tof_above_histo), 2),
        int(bib_hits_dict["alpha"][max_eff+1]), round(100*int(bib_hits_dict["alpha"][max_eff+1])/np.sum(bib_tof_above_histo), 2),
        int(bib_hits_dict["beta"][max_eff+1]), round(100*int(bib_hits_dict["beta"][max_eff+1])/np.sum(bib_tof_above_histo), 2))
plt.title(fig_title)

#plt.title(r"$\bf{CMS\,\,Phase}\bf{-2}$ Simulation Preliminary", fontsize=16, loc="left")
#plt.legend(facecolor='white', framealpha=1)
plt.show()

In [ ]:
#show plot of data and mask at right coordinate
#%matplotlib inline
"""
plot BIB data
"""
fig = plt.figure(figsize=(9,6))

bib_data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(bib_oxygen_tof_above_histo==0, bib_oxygen_tof_above_histo),
                               vmin=1, vmax=np.max(combined_tof_above_histo),
                               norm=LogNorm(), cmap=plt.get_cmap('viridis'))

"""
timewalk curve
"""
# shift timewalk curves to optimal position
diff = data["Timewalk"].iloc[-1]-cell_mask_coord_list[max_eff]
#plt.plot(data["timewalk_shifted"]-shift_offset*tof_scaling-diff, data["Qel "], label=l1, c="r")

"""
colorbar settings
"""
bib_cbar = plt.colorbar(bib_data_plot)
bib_cbar.ax.set_ylabel('BIB entries')

"""
tornado mask at max. efficiency
"""
mask = plt.pcolormesh(X_mask, Y_mask, cell_mask_bib_list[max_eff],
                      alpha=.3, cmap=plt.get_cmap('Greys_r'))

"""
plot settings
"""
plt.xlabel("Time of flight (ns)", fontsize=16)
plt.ylabel("Charge / {} ns (e-)".format(bin_width), fontsize=16)
#plt.ylim(0,80000)
plt.xlim(overlay_xedges[0], overlay_xedges[-1])
plt.grid()

fig_title = "Mask coordinate: {} ns\nPU 200: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]\nBIB: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]".format(
        round(cell_mask_coord_list[max_eff], 2),
        int(pu_hits_dict["alpha"][max_eff+1]), round(100*int(pu_hits_dict["alpha"][max_eff+1])/np.sum(pu_tof_above_histo), 2),
        int(pu_hits_dict["beta"][max_eff+1]), round(100*int(pu_hits_dict["beta"][max_eff+1])/np.sum(pu_tof_above_histo), 2),
        int(bib_hits_dict["alpha"][max_eff+1]), round(100*int(bib_hits_dict["alpha"][max_eff+1])/np.sum(bib_tof_above_histo), 2),
        int(bib_hits_dict["beta"][max_eff+1]), round(100*int(bib_hits_dict["beta"][max_eff+1])/np.sum(bib_tof_above_histo), 2))
plt.title(fig_title)

plt.show()

In [ ]:
#show plot of data and mask at right coordinate
%matplotlib inline
"""
plot PU 200 data
"""
fig = plt.figure(figsize=(9,6))

pu_data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(pu_tof_above_histo==0, pu_tof_above_histo),
                              vmin=1, vmax=np.max(combined_tof_above_histo),
                              norm=LogNorm(), cmap=plt.get_cmap('viridis'))


"""
timewalk curve
"""
# shift timewalk curves to optimal position
diff = data["Timewalk"].iloc[-1]-cell_mask_coord_list[max_eff]
#plt.plot(data["timewalk_shifted"]-shift_offset*tof_scaling-diff, data["Qel "], label=l1, c="r")

"""
colorbar settings
"""
pu_cbar = plt.colorbar(pu_data_plot)
pu_cbar.ax.set_ylabel('PU 200 entries')

"""
tornado mask at max. efficiency
"""
mask = plt.pcolormesh(X_mask, Y_mask, cell_mask_pu_list[max_eff],
                      alpha=.3, cmap=plt.get_cmap('Greys_r'))

"""
plot settings
"""
plt.xlabel("Time of flight (ns)", fontsize=16)
plt.ylabel("Charge / {} ns (e-)".format(bin_width), fontsize=16)
#plt.ylim(0,80000)
plt.xlim(overlay_xedges[0], overlay_xedges[-1])
plt.grid()

fig_title = "Mask coordinate: {} ns\nPU 200: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]\nBIB: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]".format(
        round(cell_mask_coord_list[max_eff], 2),
        int(pu_hits_dict["alpha"][max_eff+1]), round(100*int(pu_hits_dict["alpha"][max_eff+1])/np.sum(pu_tof_above_histo), 2),
        int(pu_hits_dict["beta"][max_eff+1]), round(100*int(pu_hits_dict["beta"][max_eff+1])/np.sum(pu_tof_above_histo), 2),
        int(bib_hits_dict["alpha"][max_eff+1]), round(100*int(bib_hits_dict["alpha"][max_eff+1])/np.sum(bib_tof_above_histo), 2),
        int(bib_hits_dict["beta"][max_eff+1]), round(100*int(bib_hits_dict["beta"][max_eff+1])/np.sum(bib_tof_above_histo), 2))
plt.title(fig_title)

plt.show()

# ROOT plots

In [ ]:
# erase cached settings
ROOT.gROOT.Reset()

efficiency_fname= "tornado/efficiency_root.png"
optimal_overlay_fname = "tornado/optimal_overlay_root.png"
pu_alpha_hits_root_fname = "tornado/pu_alpha_hits_root.png"
pu_beta_hits_root_fname = "tornado/pu_beta_hits_root.png"
bib_alpha_hits_root_fname = "tornado/bib_alpha_hits_root.png"
bib_beta_hits_root_fname = "tornado/bib_beta_hits_root.png"
bib_halo_hits_root_fname = "tornado/bib_halo_hits_root.png"
bib_gas_hits_root_fname = "tornado/bib_gas_hits_root.png"

In [ ]:
# put optimal overlay to 6 ns

# create root efficiency plots
plot_pu_alpha = ROOT.TGraph(len(cell_mask_coord_list),
                              np.array(cell_mask_coord_list),
                              np.array(pu_hits_dict["alpha_eff"]))
plot_pu_beta = ROOT.TGraph(len(cell_mask_coord_list),
                              np.array(cell_mask_coord_list),
                              np.array(pu_hits_dict["beta_eff"]))
plot_bib_alpha = ROOT.TGraph(len(cell_mask_coord_list),
                              np.array(cell_mask_coord_list),
                              np.array(bib_hits_dict["alpha_eff"]))
plot_bib_beta = ROOT.TGraph(len(cell_mask_coord_list),
                              np.array(cell_mask_coord_list),
                              np.array(bib_hits_dict["beta_eff"]))

# create root 2d histo for optimal chip mask overlay
plot_optimal_overlay_bib = ROOT.TH2F("Optimal overlay BIB", "Optimal overlay BIB",
               len(overlay_xedges)-1, overlay_xedges[0], overlay_xedges[-1],
               len(overlay_yedges)-1, overlay_yedges[0], overlay_yedges[-1])

plot_optimal_overlay_pu = ROOT.TH2F("Optimal overlay PU", "Optimal overlay PU",
               len(overlay_xedges)-1, overlay_xedges[0], overlay_xedges[-1],
               len(overlay_yedges)-1, overlay_yedges[0], overlay_yedges[-1])

plot_tornado_mask = ROOT.TH2F("Tornado mask", "Tornado mask",
               len(overlay_xedges)-1, overlay_xedges[0], overlay_xedges[-1],
               len(overlay_yedges)-1, overlay_yedges[0], overlay_yedges[-1])

# pu alpha hits
plot_pu_alpha_hits_root = ROOT.TH2F("PU alpha hits", "PU alpha hits",
               len(overlay_xedges)-1, overlay_xedges[0], overlay_xedges[-1],
               len(overlay_yedges)-1, overlay_yedges[0], overlay_yedges[-1])

# pu beta hits
plot_pu_beta_hits_root = ROOT.TH2F("PU beta hits", "PU beta hits",
               len(overlay_xedges)-1, overlay_xedges[0], overlay_xedges[-1],
               len(overlay_yedges)-1, overlay_yedges[0], overlay_yedges[-1])

# bib alpha hits
plot_bib_alpha_hits_root = ROOT.TH2F("BIB alpha hits", "BIB alpha hits",
               len(overlay_xedges)-1, overlay_xedges[0], overlay_xedges[-1],
               len(overlay_yedges)-1, overlay_yedges[0], overlay_yedges[-1])

# bib beta hits
plot_bib_beta_hits_root = ROOT.TH2F("BIB beta hits", "BIB beta hits",
               len(overlay_xedges)-1, overlay_xedges[0], overlay_xedges[-1],
               len(overlay_yedges)-1, overlay_yedges[0], overlay_yedges[-1])

# histo for halo only
plot_halo_root = ROOT.TH2F("BIB all halo hits", "BIB all halo hits",
               len(overlay_xedges)-1, overlay_xedges[0], overlay_xedges[-1],
               len(overlay_yedges)-1, overlay_yedges[0], overlay_yedges[-1])

# histo for gas only
plot_gas_root = ROOT.TH2F("BIB all gas hits", "BIB all gas hits",
               len(overlay_xedges)-1, overlay_xedges[0], overlay_xedges[-1],
               len(overlay_yedges)-1, overlay_yedges[0], overlay_yedges[-1])

stacked_data_mask = ROOT.THStack("stack", "Stacked histograms")

# fill up 2D histo with data
for row in range(len(overlay_xedges)-1):
    for col in range(len(overlay_yedges)-1):
            
            # fill data histo
            for i in range(int(bib_tof_above_histo[row][col])):
                plot_optimal_overlay_bib.Fill(overlay_xedges[col], overlay_yedges[row])
                
            for i in range(int(pu_tof_above_histo[row][col])):
                plot_optimal_overlay_pu.Fill(overlay_xedges[col], overlay_yedges[row])
                
            # fill pu alpha hits
            for i in range(int(pu_hits_dict["alpha_histo"][max_eff-2][row][col])):
                plot_pu_alpha_hits_root.Fill(overlay_xedges[col], overlay_yedges[row])

            # fill pu beta hits
            for i in range(int(pu_hits_dict["beta_histo"][max_eff-2][row][col])):
                plot_pu_beta_hits_root.Fill(overlay_xedges[col], overlay_yedges[row])

            # fill bib alpha hits
            for i in range(int(bib_hits_dict["alpha_histo"][max_eff-2][row][col])):
                plot_bib_alpha_hits_root.Fill(overlay_xedges[col], overlay_yedges[row])

            # fill bib beta hits
            for i in range(int(bib_hits_dict["beta_histo"][max_eff-2][row][col])):
                plot_bib_beta_hits_root.Fill(overlay_xedges[col], overlay_yedges[row])
                
            # fill bib halo hits
            for i in range(int(bib_halo_tof_above_histo[row][col])):
                plot_halo_root.Fill(overlay_xedges[col], overlay_yedges[row])

            # fill bib gas hits
            for i in range(int(bib_gas_tof_above_histo[row][col])):
                plot_gas_root.Fill(overlay_xedges[col], overlay_yedges[row])
            
            # fill mask histo
            plot_tornado_mask.Fill(overlay_xedges[col], overlay_yedges[row], 
                                   cell_mask_pu_list[max_eff-2][row, col]+cell_mask_bib_list[max_eff-2][row, col])
                
# shift timewalk curves to optimal position
diff = data["Timewalk"].iloc[-1]-cell_mask_coord_list[max_eff-2]

# tornado mask bounding curves (left, mid, right)
plot_timewalk_curves = []
plot_timewalk_curves.append(ROOT.TGraph(len(data["Timewalk"]),
                                            np.array(data["Timewalk"]-shift_offset*tof_scaling-diff),
                                            np.array(data["Qel "])))
plot_timewalk_curves.append(ROOT.TGraph(len(data["Timewalk"]),
                                            np.array(data["Timewalk"]-diff),
                                            np.array(data["Qel "])))
plot_timewalk_curves.append(ROOT.TGraph(len(data["timewalk_shifted"]),
                                           np.array(data["timewalk_shifted"]-diff),
                                           np.array(data["Qel "])))

# optimal mask location as a vertical line
plot_opt_loc = ROOT.TLine(cell_mask_coord_list[max_eff-2], -10, cell_mask_coord_list[max_eff-2], 110)

In [ ]:
# create and start editing canvas
c_pu_alpha_hits_root = ROOT.TCanvas("canvas_alpha_pu","canvas_alpha_pu")
c_pu_alpha_hits_root.Print(pu_alpha_hits_root_fname + " [")
c_pu_alpha_hits_root.Draw()
c_pu_alpha_hits_root.cd()

"""
plot data
"""
# create and start editing pad on canvas
pad_pu_alpha_hits_root = ROOT.TPad("", "",0,0,1,1)
pad_pu_alpha_hits_root.Draw()
pad_pu_alpha_hits_root.cd()
pad_pu_alpha_hits_root.SetLogz()

# draw histogram on pad
plot_pu_alpha_hits_root.Draw("colz")

# plot timewalk curves at optimal position
for plot_timewalk_curve in plot_timewalk_curves:
    plot_timewalk_curve.Draw("same")
    plot_timewalk_curve.SetLineColor(ROOT.kRed)

ROOT.gStyle.SetPalette(57)
"""
axis settings
"""
# set axis titles
plot_pu_alpha_hits_root.GetXaxis().SetTitle("Time of flight [ns]")
plot_pu_alpha_hits_root.GetYaxis().SetTitle("Charge [e-]")
plot_pu_alpha_hits_root.GetZaxis().SetTitle("PU 200 in-time hits")
# bring axis titles closer to plot (1 is ok because it is set for the widest axis label)
plot_pu_alpha_hits_root.GetYaxis().SetTitleOffset(1)
plot_pu_alpha_hits_root.GetZaxis().SetTitleOffset(1)
#pad_pu_alpha_hits_root.SetGrid()
    
"""
set plot margins
"""
# modify pad margins to fit in colorbar labels
palette = plot_pu_alpha_hits_root.GetListOfFunctions().FindObject("palette")
palette.SetX1NDC(0.81)
palette.SetX2NDC(0.86)
pad_pu_alpha_hits_root.SetLeftMargin(0.15)
pad_pu_alpha_hits_root.SetRightMargin(0.2)

# add cms title (aligns correctly if put after margin has been set)
tdrstyle.draw_CMS_text(pad_pu_alpha_hits_root)
pad_pu_alpha_hits_root.Update()

# print histo to file
c_pu_alpha_hits_root.Print(pu_alpha_hits_root_fname)
c_pu_alpha_hits_root.Print(pu_alpha_hits_root_fname + " ]")

In [ ]:
# create and start editing canvas
c_pu_beta_hits_root = ROOT.TCanvas("canvas_beta_pu","canvas_beta_pu")
c_pu_beta_hits_root.Print(pu_beta_hits_root_fname + " [")
c_pu_beta_hits_root.Draw()
c_pu_beta_hits_root.cd()

"""
plot data
"""
# create and start editing pad on canvas
pad_pu_beta_hits_root = ROOT.TPad("", "",0,0,1,1)
pad_pu_beta_hits_root.Draw()
pad_pu_beta_hits_root.cd()
pad_pu_beta_hits_root.SetLogz()

# draw histogram on pad
plot_pu_beta_hits_root.Draw("colz")

# plot timewalk curves at optimal position
for plot_timewalk_curve in plot_timewalk_curves:
    plot_timewalk_curve.Draw("same")
    plot_timewalk_curve.SetLineColor(ROOT.kRed)

ROOT.gStyle.SetPalette(57)
"""
axis settings
"""
# set axis titles
plot_pu_beta_hits_root.GetXaxis().SetTitle("Time of flight [ns]")
plot_pu_beta_hits_root.GetYaxis().SetTitle("Charge [e-]")
plot_pu_beta_hits_root.GetZaxis().SetTitle("PU 200 out-of-time hits")
# bring axis titles closer to plot (1 is ok because it is set for the widest axis label)
plot_pu_beta_hits_root.GetYaxis().SetTitleOffset(1)
plot_pu_beta_hits_root.GetZaxis().SetTitleOffset(1)
#pad_pu_beta_hits_root.SetGrid()
    
"""
set plot margins
"""
# modify pad margins to fit in colorbar labels
palette = plot_pu_beta_hits_root.GetListOfFunctions().FindObject("palette")
palette.SetX1NDC(0.81)
palette.SetX2NDC(0.86)
pad_pu_beta_hits_root.SetLeftMargin(0.15)
pad_pu_beta_hits_root.SetRightMargin(0.2)

# add cms title (aligns correctly if put after margin has been set)
tdrstyle.draw_CMS_text(pad_pu_beta_hits_root)
pad_pu_beta_hits_root.Update()

# print histo to file
c_pu_beta_hits_root.Print(pu_beta_hits_root_fname)
c_pu_beta_hits_root.Print(pu_beta_hits_root_fname + " ]")

In [ ]:
# create and start editing canvas
c_bib_alpha_hits_root = ROOT.TCanvas("canvas_alpha_bib","canvas_alpha_bib")
c_bib_alpha_hits_root.Print(bib_alpha_hits_root_fname + " [")
c_bib_alpha_hits_root.Draw()
c_bib_alpha_hits_root.cd()

"""
plot data
"""
# create and start editing pad on canvas
pad_bib_alpha_hits_root = ROOT.TPad("", "",0,0,1,1)
pad_bib_alpha_hits_root.Draw()
pad_bib_alpha_hits_root.cd()
pad_bib_alpha_hits_root.SetLogz()

# draw histogram on pad
plot_bib_alpha_hits_root.Draw("colz")

# plot timewalk curves at optimal position
for plot_timewalk_curve in plot_timewalk_curves:
    plot_timewalk_curve.Draw("same")
    plot_timewalk_curve.SetLineColor(ROOT.kRed)

ROOT.gStyle.SetPalette(57)
"""
axis settings
"""
# set axis titles
plot_bib_alpha_hits_root.GetXaxis().SetTitle("Time of flight [ns]")
plot_bib_alpha_hits_root.GetYaxis().SetTitle("Charge [e-]")
plot_bib_alpha_hits_root.GetZaxis().SetTitle("BIB in-time hits")
# bring axis titles closer to plot (1 is ok because it is set for the widest axis label)
plot_bib_alpha_hits_root.GetYaxis().SetTitleOffset(1)
plot_bib_alpha_hits_root.GetZaxis().SetTitleOffset(1)
#pad_bib_alpha_hits_root.SetGrid()
    
"""
set plot margins
"""
# modify pad margins to fit in colorbar labels
palette = plot_bib_alpha_hits_root.GetListOfFunctions().FindObject("palette")
palette.SetX1NDC(0.81)
palette.SetX2NDC(0.86)
pad_bib_alpha_hits_root.SetLeftMargin(0.15)
pad_bib_alpha_hits_root.SetRightMargin(0.2)

# add cms title (aligns correctly if bibt after margin has been set)
tdrstyle.draw_CMS_text(pad_bib_alpha_hits_root)
pad_bib_alpha_hits_root.Update()

# print histo to file
c_bib_alpha_hits_root.Print(bib_alpha_hits_root_fname)
c_bib_alpha_hits_root.Print(bib_alpha_hits_root_fname + " ]")

In [ ]:
# create and start editing canvas
c_bib_beta_hits_root = ROOT.TCanvas("canvas_beta_bib","canvas_beta_bib")
c_bib_beta_hits_root.Print(bib_beta_hits_root_fname + " [")
c_bib_beta_hits_root.Draw()
c_bib_beta_hits_root.cd()

"""
plot data
"""
# create and start editing pad on canvas
pad_bib_beta_hits_root = ROOT.TPad("", "",0,0,1,1)
pad_bib_beta_hits_root.Draw()
pad_bib_beta_hits_root.cd()
pad_bib_beta_hits_root.SetLogz()

# draw histogram on pad
plot_bib_beta_hits_root.Draw("colz")

# plot timewalk curves at optimal position
for plot_timewalk_curve in plot_timewalk_curves:
    plot_timewalk_curve.Draw("same")
    plot_timewalk_curve.SetLineColor(ROOT.kRed)

ROOT.gStyle.SetPalette(57)
"""
axis settings
"""
# set axis titles
plot_bib_beta_hits_root.GetXaxis().SetTitle("Time of flight [ns]")
plot_bib_beta_hits_root.GetYaxis().SetTitle("Charge [e-]")
plot_bib_beta_hits_root.GetZaxis().SetTitle("BIB out-of-time hits")
# bring axis titles closer to plot (1 is ok because it is set for the widest axis label)
plot_bib_beta_hits_root.GetYaxis().SetTitleOffset(1)
plot_bib_beta_hits_root.GetZaxis().SetTitleOffset(1)
#pad_bib_beta_hits_root.SetGrid()
    
"""
set plot margins
"""
# modify pad margins to fit in colorbar labels
palette = plot_bib_beta_hits_root.GetListOfFunctions().FindObject("palette")
palette.SetX1NDC(0.81)
palette.SetX2NDC(0.86)
pad_bib_beta_hits_root.SetLeftMargin(0.15)
pad_bib_beta_hits_root.SetRightMargin(0.2)

# add cms title (aligns correctly if bibt after margin has been set)
tdrstyle.draw_CMS_text(pad_bib_beta_hits_root)
pad_bib_beta_hits_root.Update()

# print histo to file
c_bib_beta_hits_root.Print(bib_beta_hits_root_fname)
c_bib_beta_hits_root.Print(bib_beta_hits_root_fname + " ]")

In [ ]:
"""
main plot with all data and mask
"""
# create and start editing canvas
c_optimal_overlay = ROOT.TCanvas("canvas","canvas")
c_optimal_overlay.Print(optimal_overlay_fname + " [")
c_optimal_overlay.Draw()
c_optimal_overlay.cd()

"""
plot data
"""
# create and start editing pad on canvas
pad_optimal_overlay = ROOT.TPad("", "",0,0,1,1)
pad_optimal_overlay.Draw()
pad_optimal_overlay.cd()
pad_optimal_overlay.SetLogz()

ex1 = ROOT.TExec('ex1','gStyle->SetPalette(kBird,0,1.0);')
ex2 = ROOT.TExec('ex2','gStyle->SetPalette(kBlueYellow,0,1.0);')
drawopt = 'colz SAME0'

# draw histogram on pad
plot_optimal_overlay_bib.Draw("colz")
ex1.Draw()
plot_optimal_overlay_bib.Draw(drawopt)
ex2.Draw()
plot_optimal_overlay_pu.Draw(drawopt)

# plot timewalk curves at optimal position
for plot_timewalk_curve in plot_timewalk_curves:
    plot_timewalk_curve.Draw("same")
    plot_timewalk_curve.SetLineColor(ROOT.kRed)

#plot_tornado_mask.Draw("colz")
#plot_tornado_mask.SetFillColorAlpha(0, 1);
#ROOT.gStyle.SetPalette(57)
"""
axis settings
"""
# set axis titles
plot_optimal_overlay_bib.GetXaxis().SetTitle("Time of flight [ns]")
plot_optimal_overlay_bib.GetYaxis().SetTitle("Charge [e-]")
#plot_optimal_overlay_bib.GetZaxis().SetTitle("Hits")
# bring axis titles closer to plot (1 is ok because it is set for the widest axis label)
plot_optimal_overlay_bib.GetYaxis().SetTitleOffset(1)
#plot_optimal_overlay_bib.GetZaxis().SetTitleOffset(1)
#pad_optimal_overlay.SetGrid()
    
"""
set plot margins
"""
# modify pad margins to fit in colorbar labels
palette = plot_optimal_overlay_bib.GetListOfFunctions().FindObject("palette")
palette.SetX1NDC(0.81)
palette.SetX2NDC(0.86)
pad_optimal_overlay.SetLeftMargin(0.15)
pad_optimal_overlay.SetRightMargin(0.2)


N = 2
histos_list = [plot_optimal_overlay_bib, plot_optimal_overlay_pu]
histo_cbar_titles = ["BIB hits", "PU 200 hits"]
for j in range(N):
    y1 = pad_optimal_overlay.GetBottomMargin()
    y2 = 1 - pad_optimal_overlay.GetTopMargin()
    x1 = pad_optimal_overlay.GetLeftMargin()
    x2 = 1 - pad_optimal_overlay.GetRightMargin()
    pvsize = (y2-y1)/float(N)
    
    gap = 0.02
    if j==0 :
        y1b = y1 + j*pvsize
    else :
        y1b = y1 + j*pvsize + gap/2.0
    
    if j==N-1 :  
        y2b = y1 + (j+1)*pvsize
    else :
        y2b = y1 + (j+1)*pvsize - gap/2.0
    
    x1b = x2 + 0.01
    x2b = x1b + 0.03
    
    palette = histos_list[j].GetListOfFunctions().FindObject('palette')
    palette.SetY1NDC(y1b)
    palette.SetY2NDC(y2b)
    palette.SetX1NDC(x1b)
    palette.SetX2NDC(x2b)
    palette.SetBorderSize(4)
    palette.SetLineColor(1)
    histos_list[j].GetZaxis().SetTitle(histo_cbar_titles[j])
    histos_list[j].GetZaxis().SetTitleOffset(1)
    histos_list[j].SetMaximum(np.max(combined_tof_above_histo))

    pFrame = ROOT.TPave(x1b,y1b,x2b,y2b,1,'NDC')
    pFrame.SetFillStyle(0)
    pFrame.SetLineColor(1)
    pFrame.SetLineWidth(1)
    pFrame.SetShadowColor(0)
    pFrame.Draw()
    
# add cms title (aligns correctly if put after margin has been set)
tdrstyle.draw_CMS_text(pad_optimal_overlay)
pad_optimal_overlay.Update()

# print histo to file
c_optimal_overlay.Print(optimal_overlay_fname)
c_optimal_overlay.Print(optimal_overlay_fname + " ]")

In [ ]:
"""
BIB halo
"""
# create and start editing canvas
c_halo_root = ROOT.TCanvas("canvas_halo","canvas_halo")
c_halo_root.Print(bib_halo_hits_root_fname + " [")
c_halo_root.Draw()
c_halo_root.cd()

"""
plot data
"""
# create and start editing pad on canvas
pad_halo_root = ROOT.TPad("", "",0,0,1,1)
pad_halo_root.Draw()
pad_halo_root.cd()
pad_halo_root.SetLogz()

# draw histogram on pad
plot_halo_root.Draw("colz")
#pad_halo_root.SetFillColorAlpha(54, 0)

#plot_tornado_mask.Draw("colz")
#plot_tornado_mask.SetFillColorAlpha(0, 1);
ROOT.gStyle.SetPalette(57)
"""
axis settings
"""
# set axis titles
plot_halo_root.GetXaxis().SetTitle("Time of flight [ns]")
plot_halo_root.GetYaxis().SetTitle("Charge [e-]")
plot_halo_root.GetZaxis().SetTitle("BIB halo hits")
# bring axis titles closer to plot (1 is ok because it is set for the widest axis label)
plot_halo_root.GetYaxis().SetTitleOffset(1)
plot_halo_root.GetZaxis().SetTitleOffset(1)
#pad_halo_root.SetGrid()
    
"""
set plot margins
"""
# modify pad margins to fit in colorbar labels
palette = plot_halo_root.GetListOfFunctions().FindObject("palette")
palette.SetX1NDC(0.81)
palette.SetX2NDC(0.86)
pad_halo_root.SetLeftMargin(0.15)
pad_halo_root.SetRightMargin(0.2)

# add cms title (aligns correctly if put after margin has been set)
tdrstyle.draw_CMS_text(pad_halo_root)
pad_halo_root.Update()

# print histo to file
c_halo_root.Print(bib_halo_hits_root_fname)
c_halo_root.Print(bib_halo_hits_root_fname + " ]")

In [ ]:
"""
BIB gas
"""
# create and start editing canvas
c_gas_root = ROOT.TCanvas("canvas_gas","canvas_gas")
c_gas_root.Print(bib_gas_hits_root_fname + " [")
c_gas_root.Draw()
c_gas_root.cd()

"""
plot data
"""
# create and start editing pad on canvas
pad_gas_root = ROOT.TPad("", "",0,0,1,1)
pad_gas_root.Draw()
pad_gas_root.cd()
pad_gas_root.SetLogz()

# draw histogram on pad
plot_gas_root.Draw("colz")
#pad_gas_root.SetFillColorAlpha(54, 0)

#plot_tornado_mask.Draw("colz")
#plot_tornado_mask.SetFillColorAlpha(0, 1);
ROOT.gStyle.SetPalette(57)
"""
axis settings
"""
# set axis titles
plot_gas_root.GetXaxis().SetTitle("Time of flight [ns]")
plot_gas_root.GetYaxis().SetTitle("Charge [e-]")
plot_gas_root.GetZaxis().SetTitle("BIB gas hits")
# bring axis titles closer to plot (1 is ok because it is set for the widest axis label)
plot_gas_root.GetYaxis().SetTitleOffset(1)
plot_gas_root.GetZaxis().SetTitleOffset(1)
#pad_gas_root.SetGrid()
    
"""
set plot margins
"""
# modify pad margins to fit in colorbar labels
palette = plot_gas_root.GetListOfFunctions().FindObject("palette")
palette.SetX1NDC(0.81)
palette.SetX2NDC(0.86)
pad_gas_root.SetLeftMargin(0.15)
pad_gas_root.SetRightMargin(0.2)

# add cms title (aligns correctly if put after margin has been set)
tdrstyle.draw_CMS_text(pad_gas_root)
pad_gas_root.Update()

# print histo to file
c_gas_root.Print(bib_gas_hits_root_fname)
c_gas_root.Print(bib_gas_hits_root_fname + " ]")

In [ ]:
# create and start editing canvas
c_efficiency = ROOT.TCanvas("canvas","canvas")
c_efficiency.Print(efficiency_fname + " [")
c_efficiency.Draw()
c_efficiency.cd()

"""
plot data
"""
# create and start editing pad on canvas
pad_alpha = ROOT.TPad("upper", "upper",0,.5,1,1)
pad_alpha.Draw()
pad_alpha.cd()

# draw alpha plot
plot_pu_alpha.Draw()
plot_bib_alpha.Draw("CP")
plot_pu_alpha.SetLineColor(ROOT.kRed)
plot_bib_alpha.SetLineColor(ROOT.kBlue)
plot_pu_alpha.SetMarkerSize(.8)
plot_pu_alpha.SetMarkerStyle(20)
plot_pu_alpha.SetMarkerColor(ROOT.kRed)
plot_bib_alpha.SetMarkerSize(.8)
plot_bib_alpha.SetMarkerStyle(20)
plot_bib_alpha.SetMarkerColor(ROOT.kBlue)

# draw optimal mask position
plot_opt_loc.Draw("CP")
plot_opt_loc.SetLineColor(ROOT.kBlack)
plot_opt_loc.SetLineStyle(2)

c_efficiency.cd()

# create and start editng pad on canvas
pad_beta = ROOT.TPad("lower", "lower",0,0,1,.5)
pad_beta.Draw()
pad_beta.cd()

# draw beta plot
plot_pu_beta.Draw()
plot_bib_beta.Draw("CP")
plot_pu_beta.SetLineColor(ROOT.kRed)
plot_bib_beta.SetLineColor(ROOT.kBlue)
plot_pu_beta.SetMarkerSize(.8)
plot_pu_beta.SetMarkerStyle(20)
plot_pu_beta.SetMarkerColor(ROOT.kRed)
plot_bib_beta.SetMarkerSize(.8)
plot_bib_beta.SetMarkerStyle(20)
plot_bib_beta.SetMarkerColor(ROOT.kBlue)

# draw optimal mask position
plot_opt_loc.Draw("CP")
plot_opt_loc.SetLineColor(ROOT.kBlack)
plot_opt_loc.SetLineStyle(2)

"""
axis settings
"""
# set axis titles
plot_pu_alpha.GetXaxis().SetTitle("Front-end clock delay setting [ns]")
plot_pu_alpha.GetYaxis().SetTitle("Efficiency [%]")
plot_pu_beta.GetXaxis().SetTitle("Front-end clock delay setting [ns]")
plot_pu_beta.GetYaxis().SetTitle("Fraction of out-of-time hits [%]")

# bring axis titles closer to plot
plot_pu_alpha.GetYaxis().SetTitleOffset(.6)
plot_pu_beta.GetYaxis().SetTitleOffset(.6)

# set manual y range
plot_pu_alpha.GetYaxis().SetRangeUser(-10, 110)
plot_pu_beta.GetYaxis().SetRangeUser(-10, 110)
plot_pu_alpha.GetXaxis().SetRangeUser(-25, 50)
plot_pu_beta.GetXaxis().SetRangeUser(-25, 50)

# set grid
#pad_alpha.SetGrid()
#pad_beta.SetGrid()

"""
set plot margins
"""
pad_alpha.SetTopMargin(0.1)

"""
cms title
"""
# add cms title (aligns correctly if put after margin has been set)
cmsText     = "CMS Phase-2";
cmsTextSize      = .95
cmsTextFont   = 61
extraText   = "Simulation Preliminary"
extraTextFont = 52
extraTextSize = 0.75*cmsTextSize

H = pad_alpha.GetWh()
W = pad_alpha.GetWw()
l = pad_alpha.GetLeftMargin()
t = pad_alpha.GetTopMargin()
r = pad_alpha.GetRightMargin()
b = pad_alpha.GetBottomMargin()
e = 0.025

pad_alpha.cd()

latex = ROOT.TLatex()
latex.SetNDC()
latex.SetTextAngle(0)
latex.SetTextColor(ROOT.kBlack)

#out of frame top left aligned
latex.SetTextFont(cmsTextFont)
latex.SetTextAlign(11)
latex.SetTextSize(cmsTextSize*t)
latex.DrawLatex(l,1-t+0.2*t,cmsText)
latex.SetTextFont(extraTextFont)
latex.SetTextAlign(11)
latex.SetTextSize(extraTextSize*t)
latex.DrawLatex(l+0.2,1-t+0.2*t, extraText) # was l+.29

pad_alpha.Update()

"""
set BRIL TDR plot style
"""
tdrstyle.setTDRStyle()

"""
add legend
"""
pad_alpha.cd()

leg_alpha = ROOT.TLegend(.6,.6, .95,.8)
leg_alpha.AddEntry(plot_pu_alpha, "PU 200","l")
leg_alpha.AddEntry(plot_bib_alpha, "BIB","l")
leg_alpha.AddEntry(plot_opt_loc, "Optimal chip efficiency at {} ns".format(
    int(cell_mask_coord_list[max_eff-2])),"l")
leg_alpha.SetTextFont(42)
leg_alpha.SetBorderSize(0)
leg_alpha.SetFillStyle(0)
plot_pu_alpha.GetListOfFunctions().Add(leg_alpha)
leg_alpha.Draw("CP")

c_efficiency.cd()
pad_beta.cd()

leg_beta = ROOT.TLegend(.6,.3, .95,.5)
leg_beta.AddEntry(plot_pu_beta, "PU 200","l")
leg_beta.AddEntry(plot_bib_beta, "BIB","l")
leg_beta.AddEntry(plot_opt_loc, "Optimal chip efficiency at {} ns".format(
        int(cell_mask_coord_list[max_eff-2])),"l")
leg_beta.SetTextFont(42)
leg_beta.SetBorderSize(0)
leg_beta.SetFillStyle(0)
plot_pu_beta.GetListOfFunctions().Add(leg_beta)
leg_beta.Draw("CP")

"""
save plot to file
"""
c_efficiency.Print(efficiency_fname)
c_efficiency.Print(efficiency_fname + " ]")

In [ ]:
#hf_bib_carbon.close()
#hf_bib_hydrogen.close()
#hf_pu.close()